In [4]:
 import bct
import pandas as pd
import numpy as np
import os
import sys
import seaborn as sns
import pingouin
import matplotlib.pyplot as plt
import warnings
sys.path.append("/scratch/connectome/TVB/TVB_RESEARCH")
sys.path
import utils #now imports becasue it looks at the last added sys path

import warnings
warnings.filterwarnings('ignore')
import time
from tqdm import tqdm
import argparse


#parser = argparse.ArgumentParser(description = "argparse")
#
#parser.add_argument('--threshold', type = float, help = "threshold to use")
#
#args = parser.parse_args()

class arg():
    def __init__(self):
        self.threshold = 0.1
args = arg()   
data_dir = '/storage/bigdata/ABCD/TVB/data'


dataset = utils.load_data.ABCDDataset(data_dir)  #이제 된다
subjects = dataset.list_subjects()
meta_data = utils.load_metadata.metadata(data_dir)


start = time.time()
warnings.filterwarnings('once') #여기서는 한번만 나오게 하기 (여러번 돌아가면서 계속 뜨는 것은 싫다)

###how much to keep
sample_size = len(subjects) #if you wanna run all
threshold = args.threshold

##scalar meausres correlation
sample_sub = dataset.load_sc(subjects[0])
data, names = utils.conn_measures.compute_bct_UW(sample_sub).scalar_properties()

num_sub = len(subjects) #8238
num_measures = len(names)
scalar_bct_results = np.zeros((min(num_sub,sample_size),num_measures))
 
###SC + SCALAR (vector, matrix는 할지말지, 그리고 한다면 언제할지는 모르겠다)
warnings.filterwarnings("ignore")
for i,sub in enumerate(subjects[:sample_size]):#tqdm(subjects[:sample_size])):
    sub_sc = dataset.load_sc(sub)
    #do FC too!!!
    print("doing {}th subject".format(i))
    matrix_loaded = utils.conn_measures.compute_bct_UW(sub_sc, threshold=threshold)
    if matrix_loaded.usability_check_density_based() != True:
        print("{} failed".format(sub))
        continue #if usability check is not satisfied, skip this iteration
    data, names = matrix_loaded.scalar_properties()
    scalar_bct_results[i] = np.array(list(data.values()))

total_scalar_results = pd.DataFrame(scalar_bct_results, index = subjects[:sample_size], columns = names)
end = time.time()
print("run_time(s) :", end-start)


BCT_result_save_dir = "/storage/bigdata/ABCD/TVB/BCT_results/SC_ses_2/"

total_scalar_results.to_csv(BCT_result_save_dir+"scalar_results_threshold_{}.csv".format(threshold))
pd.read_csv(BCT_result_save_dir+"scalar_results_threshold_{}.csv".format(threshold), index_col=0) #use the first column as the rows
os.listdir(BCT_result_save_dir)

/scratch/connectome/TVB/TVB_RESEARCH/utils/conn_measures.py:24: UserWarning: no thresholding will be used please check if this is what you want :)!
  warnings.warn("no thresholding will be used please check if this is what you want :)!")


no thresholding will be used please check if this is what you want :)!


/home/connectome/dyhan316/.local/lib/python3.8/site-packages/numpy/core/_asarray.py:102: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order)


subject passed the test!
NDARINVRERRA7FDfailed
subject passed the test!
subject passed the test!


KeyboardInterrupt: 

In [ ]:
for i,sub in enumerate(subjects[:sample_size]): #tqdm(subjects[:sample_size])): #tqdm not used becuase if loop breaks during tqdm tqdm breaks
    sub_sc = dataset.load_sc(sub)
    #do FC too!!!
    matrix_loaded = utils.conn_measures.compute_bct_UW(sub_sc, threshold=threshold)
    if matrix_loaded.usability_check_density_based() != True:
        print("{}failed".format(sub))
		continue #if usability check is not satisfied, skip this iteration
    data, names = matrix_loaded.scalar_properties()
    scalar_bct_results[i] = np.array(list(data.values()))



In [19]:
count = 0
modular_structures = np.zeros(5)
matrix_loaded = utils.conn_measures.compute_bct_UW(dataset.load_sc(subjects[4]), threshold=threshold)
print(matrix_loaded.usability_check_density_based())
while modular_structures.max() != 5:
    modular_structures, modularities = bct.community_louvain(matrix_loaded.mat)
    count+=1
    print(count)
    if count > 1000: #MUST BE CHANGED
        if (np.max(modular_structures) == 6) or (np.max(modular_structures)==4):
            break
print(modular_structures)

KeyboardInterrupt: 

In [20]:
subjects[4]

'NDARINVFKFN7X0T'

In [24]:
utils.conn_measures.compute_bct_UW(dataset.load_sc(subjects[3]),threshold)

In [25]:
#아!!! 4번쩨가 안되는 이유가, __init__예가 하면서 돌려야하는 것이 있는데, 이게 그 작동하면 안된느 것이기 때문!
#따라서, __init__에서 정의하기 전에, 먼저 check usaiblity본 후에, usability가 check되었을때만 밑에 하도록 하기
#단, 만약 usability check 통과해도 작동을 안하는 것이 된다면,... => 이거는 조금있다가 더 생각해보자